## Census Block Group to ZCTA crossmapping operation

Certain datasets are captured at highly granular resolution. In those cases, the data may need additional features to ask questions at a broader geographic scale though smaller than State scale.

In this exercise, we will need to create a table that crossmaps between the US Census Block Group ID and geometry with the US Zip Code Tabulation Area (ZCTA) ID and geometry.

The source data will be obtained from these steps:

1) Go to the US Census website: https://www.census.gov/data/data-tools.html

2) Click on the "Explore data" menu, then choose "Data Tools and Apps"

3) Click on "Explore Census Data"; it will take you to https://data.census.gov/cedsci/

4) Search for "ZCTA shapefile

5) In the 'Geographic Products available for ZCTAs' area, click on "TIGER/Line Shapefiles"
    - https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html
    
6) Click on "FTP Archive" under the Download section
    - It will take you here: https://www2.census.gov/geo/tiger/TIGER2019/

7) Click on "ZCTA5/"
8) Download and unzip the "tl_2019_us_zcta510.zip"

9) Go back and click on "BG/"
10) Download and unzip the Block group shapefile for your place of interest. The number system corresponds to the State ID.
    - For example, download "tl_2019_72_bg.zip"


In [1]:
import geopandas as gpd
import pandas as pd
import os
%matplotlib inline

In [2]:
import urllib as urllib2
import wget
import bz2
import zipfile
import shutil
import matplotlib.pyplot as plt

In [3]:
# folder prep
if not os.path.exists(os.path.join(os.getcwd(), 'Census_2010_bg_files')):
    os.mkdir(os.path.join(os.getcwd(), 'Census_2010_bg_files'))

In [4]:
def getCensusDataSets(domain='https://www2.census.gov/geo/tiger/TIGER2019/BG',
                      fname='tl_2019_{0}_bg'.format('53'), # default example, Washington state
                      dest_folder=os.path.join(os.getcwd(), 'Census_2010_bg_files')):

    # generate source and local filepath
    filename = fname+'.zip'
    fileurl = os.path.join(domain, filename)
    
    # generate the destination folder
    # replace if it exists
    if os.path.exists(os.path.join(dest_folder, fname)):
        shutil.rmtree(os.path.join(dest_folder, fname))
            
    # download zipfile
    ping = urllib2.request.urlopen(os.path.join(domain, filename))
    if ping.getcode() != 404:
        wget.download(fileurl, out=dest_folder)

    # generate the destination folder
    os.mkdir(os.path.join(dest_folder,fname))

    # extract to destination folder
    with zipfile.ZipFile(os.path.join(dest_folder, filename), 'r') as zip_ref:
        zip_ref.extractall(os.path.join(os.getcwd(),dest_folder, fname))

    # remove zip file
    os.remove(os.path.join(dest_folder, filename))
    return(os.path.join(dest_folder, fname, fname+'.shp'))

In [5]:
#%%time
## get the 5-digit ZCTA shapefile (501 MB file read by wget)

# zcta = getCensusDataSets(domain='https://www2.census.gov/geo/tiger/TIGER2019/ZCTA5',
#                          fname='tl_2019_us_zcta510',
#                          dest_folder=os.path.join(os.getcwd(), 'Census_2010_zcta5_files'))

In [6]:
%%time
# read in the zcta dataset
zcta = gpd.read_file(os.path.join(os.getcwd(),'../Downloads/tl_2019_us_zcta510/tl_2019_us_zcta510.shp'))

print(zcta.shape)
zcta.head()

(33144, 10)
CPU times: user 16.4 s, sys: 1.49 s, total: 17.9 s
Wall time: 18.6 s


,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,43451,43451,B5,G6350,S,63484186,157689,+41.3183010,-083.6174935,"POLYGON ((-83.708733 41.327326, -83.708147 41...."
1,43452,43452,B5,G6350,S,121522304,13721730,+41.5157923,-082.9809454,"POLYGON ((-83.086978 41.537796, -83.0825629999..."
2,43456,43456,B5,G6350,S,9320975,1003775,+41.6318300,-082.8393923,"(POLYGON ((-82.835577 41.710823, -82.83515 41...."
3,43457,43457,B5,G6350,S,48004681,0,+41.2673301,-083.4274872,"POLYGON ((-83.49650299999999 41.253708, -83.48..."
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((-83.222292 41.531025, -83.2222819999..."


In [ ]:
def crossmap_BG_to_ZCTA(poly_bg, poly_zcta, outshapefilepath, uid='GEOID',
                        zip_code_fieldname='ZCTA5CE10'):
    
    """
    poly_bg (geodataframe): geodataframe of the Census Block group file
    poly_zcta (geodataframe): geodataframe of the ZCTA polygon file
    outshapefilepath (dir): the output diretory and name for the shapefile; will also be used for the csv output
    uid (str): GEOID or an equivalent to recognize unique shapes
    zip_code_fieldname (str): ZCTA5CE10 or equivalent to recognize ZIP Codes or ZCTA spatial units
    
    return: 1) mappings as a shapefile and 2) mappings as a csv table
    """

    """
    1) identify BGs with geometry that fit within ZCTAs
    """
    poly_bg['centroid'] = poly_bg.centroid
    bg_within = gpd.sjoin(poly_bg.set_geometry('geometry'), poly_zcta, 
                          how='inner', op='within').drop('index_right', axis=1)
    bg_within = bg_within.assign(overlay='geometry within ZCTA')

    print(bg_within.shape)

    """
    2) identify BGs that intersect by polygon centroid
    """
    # extract block groups that could not be joined with within method
    bg2 = poly_bg[~poly_bg['GEOID'].isin(bg_within['GEOID'])]
    bg_intersects = gpd.sjoin(bg2.set_geometry('centroid'), poly_zcta, 
                              how='inner', op='intersects').drop('index_right', axis=1)\
        .assign(overlay='centroid intersects ZCTA')

    print(bg_intersects.shape)

    """
    3) identify BGs that intersect with one zcta by polygon geometry
    """
    # extract block groups that could not be joined with centroid intersection method
    bg3 = bg2.loc[~bg2['GEOID'].isin(bg_intersects['GEOID']),:]
    bg_g_left = gpd.sjoin(bg3.set_geometry('geometry'), poly_zcta, 
                          how='inner', op='intersects').drop('index_right', axis=1)
    mappings = bg_g_left.groupby(uid)[zip_code_fieldname].nunique()
    bg_g_left1 = bg_g_left.loc[bg_g_left['GEOID'].isin(mappings[mappings==1].dropna().index),:]\
        .assign(overlay='geometry intersects a single ZCTA')

    print(bg_g_left1.shape)

    """
    4) identify BGs with multiple ZCTA by geometry intersection, but 0 ZCTA by centroid intersection
    """
    bg4 = bg3.loc[~bg3['GEOID'].isin(bg_g_left1['GEOID']),:]
    bg_c_left = gpd.sjoin(bg4.set_geometry('centroid'), poly_zcta,
                          how='left', op='intersects').drop('index_right', axis=1)
    bg_c_left1 = bg_c_left[bg_c_left['GEOID'].isin(mappings[mappings>1].dropna().index)]\
        .assign(overlay='geometry intersects multiple ZCTA, centroid intersects 0 ZCTA')

    print(bg_c_left1.shape)

    """
    5) merge the spatial joins, report, then export
    """
    bg_mapped = pd.concat([bg_within, bg_intersects, bg_g_left1, bg_c_left1], axis=0)

    # coverage
    print('geometry within: {0} ({1}%)'\
          .format(bg_within.shape[0], round(bg_within.shape[0]/poly_bg.shape[0]*100,5)))
    
    print('centroid intersects: {0} ({1}%)'\
          .format(bg_intersects.shape[0], round(bg_intersects.shape[0]/poly_bg.shape[0]*100, 5)))
    
    print('geometry intersects 1 ZCTA: {0} ({1}%)'\
          .format(bg_g_left1.shape[0], round(bg_g_left1.shape[0]/poly_bg.shape[0]*100, 5)))
    
    print('geometry intersects multiple ZCTA, centroid intersects 0 ZCTA: {0} ({1}%)'\
          .format(bg_c_left1.shape[0], round(bg_c_left1.shape[0]/poly_bg.shape[0]*100, 5)))
    
    print('total coverage: {0}/{1}'\
          .format(bg_mapped[uid].isin(poly_bg[uid]).sum()[0], poly_bg[uid].nunique()[0]))

    # create directory if not exist
    if not os.path.exists(os.path.abspath(os.path.dirname(outshapefilepath))):
        os.mkdir(os.path.abspath(os.path.dirname(outshapefilepath)))

    # export to shapefiles (replace existing shapefile)
    bg_mapped.drop('centroid', axis=1).to_file(outshapefilepath)

    # export to dataframe
    bg_mapped.drop(['geometry','centroid'], axis=1)\
        .to_csv(outshapefilepath.replace('.shp','.csv'), header=True, index=False)
    return(bg_mapped)

In [ ]:
def renderMap(gdf, outfilepath, scale_variable, plot_variable):
    
    "examine all census block group plots"
    # create directory if not exist
    if not os.path.exists(os.path.abspath(os.path.dirname(outfilepath))):
        os.mkdir(os.path.abspath(os.path.dirname(outfilepath)))
    
    # add county reference boundaries
    ax=gdf.dissolve(by='COUNTYFP').reset_index()\
        .plot(color=None, ec='black', figsize=(15,15), zorder=10, alpha=0.2)
    
    # plot the overlays
    gdf.plot('overlay', ec=None, alpha=1, zorder=1, legend=True, ax=ax, 
             legend_kwds={'loc': 'lower right', 'ncol':4, 'fontsize':9})

    plt.savefig(outfilepath, dpi=300)

## Do this for the State of Washington

In [ ]:
WA_cbg = getCensusDataSets(fname='tl_2019_{0}_bg'.format('53'), # default example, Washington state
                           dest_folder=os.path.join(os.getcwd(), 'Census_2010_bg_files'))

#Washington
bg = gpd.read_file(WA_cbg)

WA_bg_mapped = crossmap_BG_to_ZCTA(poly_bg = bg,
                                   poly_zcta = zcta.loc[:,['GEOID10','ZCTA5CE10','geometry']],
                                   outshapefilepath=os.path.join(os.getcwd(),'cbg_zcta_crossmap',
                                                                 'tl_2019_{0}_bg'.format('53'),
                                                                 'tl_2019_{0}_bg.shp'.format('53')),
                                   uid=['GEOID'],
                                   zip_code_fieldname=['ZCTA5CE10'])

WA_bg_mapped.head()

In [ ]:
renderMap(gdf=WA_bg_mapped,
          outfilepath=os.path.join(os.getcwd(),'cbg_zcta_crossmap',
                                   'tl_2019_{0}_bg'.format('53'),
                                   'tl_2019_{0}_bg.png'.format('53')),
          scale_variable='ZCTA5CE10',
          plot_variable='overlay')

## Do this for the State of Maryland

In [ ]:
MD_cbg = getCensusDataSets(fname = 'tl_2019_{0}_bg'.format('24')) #Maryland
bg = gpd.read_file(MD_cbg)

MD_bg_mapped = crossmap_BG_to_ZCTA(poly_bg = bg,
                                   poly_zcta = zcta.loc[:,['GEOID10','ZCTA5CE10','geometry']],
                                   outshapefilepath=os.path.join(os.getcwd(),'cbg_zcta_crossmap','MA_cbg.shp'),
                                   uid=['GEOID'],
                                   zip_code_fieldname=['ZCTA5CE10'])

MD_bg_mapped.head()

In [ ]:
renderMap(gdf=MD_bg_mapped,
          outfilepath=os.path.join(os.getcwd(),'cbg_zcta_crossmap',
                                   'tl_2019_{0}_bg'.format('24'),
                                   'tl_2019_{0}_bg.png'.format('24')),
          scale_variable='ZCTA5CE10',
          plot_variable='overlay')

## Do this for the State of New York

In [ ]:
NY_cbg = getCensusDataSets(fname = 'tl_2019_{0}_bg'.format('36')) #New York
bg = gpd.read_file(NY_cbg)

NY_bg_mapped = crossmap_BG_to_ZCTA(poly_bg = bg,
                                   poly_zcta = zcta.loc[:,['GEOID10','ZCTA5CE10','geometry']],
                                   outshapefilepath=os.path.join(os.getcwd(),'cbg_zcta_crossmap','NY_cbg.shp'),
                                   uid=['GEOID'],
                                   zip_code_fieldname=['ZCTA5CE10'])

NY_bg_mapped.head()

In [ ]:
renderMap(gdf=NY_bg_mapped,
          outfilepath=os.path.join(os.getcwd(),'cbg_zcta_crossmap',
                                   'tl_2019_{0}_bg'.format('36'),
                                   'tl_2019_{0}_bg.png'.format('36')),
          scale_variable='ZCTA5CE10',
          plot_variable='overlay')

## Do this for the State of California

In [ ]:
CA_cbg = getCensusDataSets(fname = 'tl_2019_{0}_bg'.format('06')) #California
bg = gpd.read_file(CA_cbg)

CA_bg_mapped = crossmap_BG_to_ZCTA(poly_bg = bg,
                                   poly_zcta = zcta.loc[:,['GEOID10','ZCTA5CE10','geometry']],
                                   outshapefilepath=os.path.join(os.getcwd(),'cbg_zcta_crossmap','CA_cbg.shp'),
                                   uid=['GEOID'],
                                   zip_code_fieldname=['ZCTA5CE10'])

CA_bg_mapped.head()

In [ ]:
renderMap(gdf=CA_bg_mapped,
          outfilepath=os.path.join(os.getcwd(),'cbg_zcta_crossmap',
                                   'tl_2019_{0}_bg'.format('06'),
                                   'tl_2019_{0}_bg.png'.format('06')),
          scale_variable='ZCTA5CE10',
          plot_variable='overlay')

In [ ]:
## remaining states queued for crossmapping

# TN
# TX
# KY
# AK
# NJ
# NH
# CT
# MA
# OR
# IN
# NC
# CA
# UT

In [ ]:
TN_cbg = getStateCBG(fname = 'tl_2019_{0}_bg'.format('47')) #Tennessee
# TX_cbg = getStateCBG(fname = 'tl_2019_{0}_bg'.format('48')) #Texas
# KY_cbg = getStateCBG(fname = 'tl_2019_{0}_bg'.format('21')) #KY
# AK_cbg = getStateCBG(fname = 'tl_2019_{0}_bg'.format('05')) #Arkansas
# NJ_cbg = getStateCBG(fname = 'tl_2019_{0}_bg'.format('34')) #New Jersey
# NH_cbg = getStateCBG(fname = 'tl_2019_{0}_bg'.format('33')) #New Hampshire
# CT_cbg = getStateCBG(fname = 'tl_2019_{0}_bg'.format('09')) #Conneticut
#MA_cbg = getStateCBG(fname = 'tl_2019_{0}_bg'.format('25')) #Massachusetts